In [ ]:
"""
Compute area of the drainage basins

"""

In [ ]:
import xarray as xr
from tqdm.notebook import tqdm
import multimelt.useful_functions as uf


In [ ]:
home_path = '/bettik/burgardc/'
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'


In [ ]:
map_lim = [-3000000,3000000]


file_mask_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
file_mask_orig_cut = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)

ice_thickness = file_mask_orig_cut['thickness']

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
mask_basins = xr.open_dataset(inputpath_data + 'Mask_Iceshelf_4km_IMBIE_withNisf.nc')

In [ ]:
grid_cell_area_file = xr.open_dataset(inputpath_data+'gridarea_ISMIP6_AIS_4000m_grid.nc').sel(x=file_isf.x,y=file_isf.y)
true_grid_cell_area = grid_cell_area_file['cell_area'].drop('lon').drop('lat')

In [ ]:
grounded_msk03 = file_isf['ground_mask'].where(file_isf['ground_mask']==0,3)
grounded_msk = (grounded_msk03.where(grounded_msk03!=3,1)-1)*-1

In [ ]:
grounded_basin = mask_basins['Iceshelf'].where(grounded_msk == 0)

map_to_plot = file_isf['ISF_mask'] * np.nan
map_to_plot2 = file_isf['ISF_mask'] * np.nan
grounded_basin_kisf = file_isf['ISF_mask'] * np.nan

for kisf in tqdm(file_isf['Nisf']): #
    other_Nisf = mask_basins['ID_IMBIE'].sel(Nisf=kisf)
    #print(kisf.values,file_isf['isf_name'].sel(Nisf=kisf).values, other_Nisf.values)
    grounded_basin_kisf = grounded_basin_kisf.where(grounded_basin != other_Nisf, kisf)
map_to_plot_all = map_to_plot.combine_first(map_to_plot2)
grounded_basin_kisf = grounded_basin_kisf.where(np.isfinite(grounded_basin_kisf), 76)

In [ ]:
total_grounded_area = (np.isfinite(grounded_basin).astype(int) * true_grid_cell_area).sum()
total_grounded_volume = (np.isfinite(grounded_basin).astype(int) * true_grid_cell_area * ice_thickness).sum()

In [ ]:
drainage_area = ((grounded_basin_kisf == 10).astype(int) * true_grid_cell_area).sum()
drainage_volume = ((grounded_basin_kisf == 10).astype(int) * true_grid_cell_area * ice_thickness).sum()

In [ ]:
drainage_area / total_grounded_area

In [ ]:
drainage_volume / total_grounded_volume

In [ ]:
np.isfinite(grounded_basin).plot()